In [1]:
import numpy as np
import pandas as pd
import scanpy as sc
import h5py

# file = h5py.File('C:/Users/Katerchen/Documents/Code/master_thesis/DrVAE-master/workspace/datafiles/CTRPv2+L1000_FDAdrugs6h_v2.1.h5', 'r')
# data = list(file.keys())

In [2]:
def load_from_HDF(fname):
    """Load data from a HDF5 file to a dictionary."""
    data = dict()
    with h5py.File(fname, 'r') as f:
        for key in f:
            data[key] = np.asarray(f[key])
            if isinstance(data[key][0], np.bytes_):
                data[key] = data[key].astype(str)
            # print(key + ":", f[key])
    return data


In [3]:
data_hdf = load_from_HDF('C:/Users/Katerchen/Documents/Code/master_thesis/DrVAE-master/workspace/datafiles/CTRPv2+L1000_FDAdrugs6h_v2.1.h5')

In [21]:
data_hdf.keys()

dict_keys(['drug_drug', 'drug_m', 'drug_threshold', 'labeled_pert_cellid', 'pair_cid', 'pair_conc', 'pair_drug', 'pair_dur', 'pair_m', 'pair_s', 'pair_tid', 'pair_x1', 'pair_x2', 'pair_y', 'pair_ycont', 'sing_cid', 'sing_s', 'sing_tid', 'sing_x1', 'sing_y', 'sing_ycont'])

In [45]:
# Dictionary to store DataFrames
dataframes_dict = {}

# Iterate through the dictionary
for key, value in data_hdf.items():
    # Create a DataFrame from the dictionary value
    df = pd.DataFrame(value)
    
    # Store the DataFrame in the dataframes_dict with the corresponding key
    dataframes_dict[key] = df

In [49]:
# show shapes of the different dataframes in the dictionary
for i in dataframes_dict:
    print(i, dataframes_dict[i].shape)

drug_drug (46, 1)
drug_m (46, 53)
drug_threshold (46, 1)
labeled_pert_cellid (52, 1)
pair_cid (5377, 1)
pair_conc (5377, 1)
pair_drug (5377, 1)
pair_dur (5377, 1)
pair_m (5377, 53)
pair_s (5377, 1)
pair_tid (5377, 1)
pair_x1 (5377, 973)
pair_x2 (5377, 973)
pair_y (5377, 1)
pair_ycont (5377, 1)
sing_cid (927, 1)
sing_s (927, 1)
sing_tid (927, 1)
sing_x1 (927, 973)
sing_y (927, 46)
sing_ycont (927, 46)


In [ ]:
# put drug_drug, drug_m and drug_threshold together

drug_df = pd.DataFrame(columns=dataframes_dict['drug_drug'].columns)
drug_df = drug_df.append(dataframes_dict['drug_drug'])

In [ ]:
# combine pair_cid, pair_conc, pair_drug, pair_dur, pair_m, pair_s, pair_tid, pair_x1, pair_x2, pair_y, pair_ycont together in a dataframe
pair_df = pd.DataFrame(columns=dataframes_dict['pair_cid'].columns)

for i in dataframes_dict:
    if i.startswith('pair'):
        pair_df = pair_df.append(dataframes_dict[i])
        
pair_df.shape

In [ ]:
sing_df = pd.DataFrame(columns=dataframes_dict['sing_cid'].columns)

for i in dataframes_dict:
    if i.startswith('sing'):
        sing_df = sing_df.append(dataframes_dict[i])
        
sing_df.shape